In [2]:
import os 
import json 
import pandas as pd 
import traceback

In [10]:
KEY=os.getenv('OPENAI_API_KEY')

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.chat_models import ChatOpenAI

In [12]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [20]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain, LLMChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [16]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [17]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [18]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [21]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [22]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [23]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [24]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [34]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], 
                                          output_variables=["quiz", "review"], verbose=True)

In [27]:
file_path=r"/llm-gen-ai-project/data.txt"

In [29]:
with open(file_path, "r") as file:
    TEXT=file.read()

In [30]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[7][8]

From a theoreti

In [33]:
# Serialize the Python dictionary into a JSON formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [35]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"


In [36]:
# how to setup token usage tracking in Langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field

In [37]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1518
Prompt Tokens:1018
Completion Tokens:500
Total Cost:0.0025269999999999997


In [41]:
quiz=response["quiz"]

In [44]:
quiz = json.loads(quiz)

In [45]:
# data frame
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        f"{option}: {option_value}"
        for option, option_value in value["options"].items()
    )
    correct = value["correct"]
    quiz_table_data.append({ "MCQ": mcq, "Choices": options, "Correct": correct })

In [46]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: A field of study in artificial intelligence concerned with the development and study of statistical algorithms | b: A field of study in biology concerned with plant growth | c: A field of study in history concerned with ancient civilizations | d: A field of study in chemistry concerned with chemical reactions',
  'Correct': 'a'},
 {'MCQ': 'Which field does machine learning find applications in?',
  'Choices': 'a: Natural language processing, computer vision, and speech recognition | b: Cooking and baking | c: Fishing and hunting | d: Dancing and singing',
  'Correct': 'a'},
 {'MCQ': 'What is the mathematical foundation of machine learning?',
  'Choices': 'a: Calculus | b: Algebra | c: Mathematical optimization (mathematical programming) methods | d: Geometry',
  'Correct': 'c'},
 {'MCQ': 'What is data mining focused on?',
  'Choices': 'a: Exploratory data analysis (EDA) through supervised learning | b: Exploratory data analysis (ED

In [48]:
quiz = pd.DataFrame(quiz_table_data)

In [50]:
# convert data on CSV file
quiz.to_csv("ML_quiz.csv", index=False)



In [55]:
from datetime import datetime
datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2024-05-02 22:34:20'